**Overview:**
The AmesHousing dataset contains data on residential properties sold in Ames, Iowa, between 2006 and 2010. It includes 2,930 observations and 82 variables related to home features, quality ratings, and sale information. The dataset is used for educational purposes, particularly in regression modeling and home price prediction.

Variables:
The dataset includes variables for:

**Property details:** MS SubClass, MS Zoning, Lot Area, Street, Alley

**Building features:** Overall Qual, Year Built, Roof Style, Foundation

**Living space and amenities:** Gr Liv Area, Bsmt Qual, Fireplaces, Garage Type,

**Pool Area
Sale information:** SalePrice, Sale Type, Sale Condition, Mo Sold, Yr Sold

Variables are a mix of nominal, ordinal, discrete, and continuous types.

**Dataset link** - https://github.com/leontoddjohnson/datasets/blob/main/data/ames.csv

**Dataset Documentation** - https://jse.amstat.org/v19n3/decock/DataDocumentation.txt

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRBmVxZInzHUScgPLUBhJ1fTxxfMAtNplVh84LGp4uzvIidFWOvKTVk2xzhBs0eBIQCYodo3v8oeUgU/pub?output=csv'

In [3]:
data = pd.read_csv(url)

data.head()

,Order,PID,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,...,Pool.Area,Pool.QC,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [6]:
!pip install dash-bootstrap-components


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 4.8 MB/s eta 0:00:00


In [7]:
import dash_bootstrap_components as dbc


In [8]:
# @title My Web App: Ames Housing Analysis

# Required Libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc

# Load data from Google Sheets (public CSV link)
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRBmVxZInzHUScgPLUBhJ1fTxxfMAtNplVh84LGp4uzvIidFWOvKTVk2xzhBs0eBIQCYodo3v8oeUgU/pub?output=csv'
data = pd.read_csv(url)

# Verify the data
print(data.head())

# Initialize the Dash app with a Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Ames Housing Dashboard"

# Layout of the app
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col(html.H1("Ames Housing Market Analysis",
                        className="text-center text-light mb-4"), width=12)
    ]),

    # Filters
    dbc.Row([
        dbc.Col([
            html.Label("Filter by Neighborhood", className="text-light"),
            dcc.Dropdown(
                id='neighborhood-filter',
                options=[{'label': n, 'value': n} for n in data['Neighborhood'].unique()],
                value=data['Neighborhood'].unique().tolist(),
                multi=True,
                style={'marginBottom': '20px'}
            ),
        ], width=4),

        dbc.Col([
            html.Label("Filter by Year Sold", className="text-light"),
            dcc.Slider(
                id='year-slider',
                min=data['Yr.Sold'].min(),
                max=data['Yr.Sold'].max(),
                value=data['Yr.Sold'].min(),
                marks={str(year): str(year) for year in data['Yr.Sold'].unique()},
                step=None,
                tooltip={"placement": "bottom", "always_visible": True},
            )
        ], width=8),
    ], className="mb-4"),

    # Main Visualizations
    dbc.Row([
        dbc.Col(dcc.Graph(id='scatter-plot'), width=6),
        dbc.Col(dcc.Graph(id='bar-chart'), width=6),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(id='time-series-plot'), width=6),
        dbc.Col(dcc.Graph(id='heatmap'), width=6),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id='histogram'), width=6),
        dbc.Col(dcc.Graph(id='sunburst-chart'), width=6),
    ], className="mb-4"),

    # Footer
    dbc.Row([
        dbc.Col(html.P("Data Source: Ames Housing Dataset - Connected to Google Sheets",
                       className="text-center text-muted"), width=12)
    ])
], fluid=True, style={'backgroundColor': '#2C2C2C', 'padding': '20px'})


# Callbacks for updating the visualizations
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-chart', 'figure'),
     Output('time-series-plot', 'figure'),
     Output('heatmap', 'figure'),
     Output('histogram', 'figure'),
     Output('sunburst-chart', 'figure')],
    [Input('neighborhood-filter', 'value'),
     Input('year-slider', 'value')]
)
def update_charts(selected_neighborhoods, selected_year):
    # Filter data
    try:
        filtered_data = data[(data['Neighborhood'].isin(selected_neighborhoods)) & (data['Yr.Sold'] == selected_year)]
        if filtered_data.empty:
            raise ValueError("No data available for the selected filters.")
    except Exception as e:
        print(f"Data filtering error: {e}")
        return [{}] * 6  # Return empty figures to prevent app crash

    # Scatter Plot
    scatter_fig = px.scatter(
        filtered_data, x="Gr.Liv.Area", y="SalePrice", color="Overall.Qual",
        title="Sale Price vs. Living Area",
        labels={"Gr.Liv.Area": "Living Area (sq ft)", "SalePrice": "Sale Price ($)"},
        color_continuous_scale="Viridis"
    )

    # Bar Chart with Dynamic Insight
    avg_price = filtered_data.groupby("Neighborhood")["SalePrice"].mean().reset_index()
    max_neighborhood = avg_price.loc[avg_price['SalePrice'].idxmax()]  # Find the highest sale price

    bar_fig = px.bar(
        avg_price, x="Neighborhood", y="SalePrice",
        title="Average Sale Price by Neighborhood",
        color="SalePrice",
        color_continuous_scale="Blues"
    )

    # Add an annotation for the highest sale price
    bar_fig.add_annotation(
        x=max_neighborhood['Neighborhood'],
        y=max_neighborhood['SalePrice'],
        text=f"Highest: {max_neighborhood['Neighborhood']} (${max_neighborhood['SalePrice']:,.0f})",
        showarrow=True,
        arrowhead=2,
        ax=-50,
        ay=-50,
        font=dict(color="black", size=12),
        bgcolor="yellow",
        bordercolor="black",
        borderwidth=1
    )

    # Time-Series Plot
    time_data = data.groupby("Yr.Sold")["SalePrice"].mean().reset_index()
    time_fig = px.line(
        time_data, x="Yr.Sold", y="SalePrice",
        title="Average Sale Price Over Time"
    )

    # Heatmap
    corr_data = data[['SalePrice', 'Gr.Liv.Area', 'Overall.Qual', 'Garage.Cars', 'Year.Built']].corr()
    heatmap_fig = px.imshow(
        corr_data, text_auto=True, color_continuous_scale="Viridis",
        title="Correlation Heatmap"
    )

    # Histogram
    hist_fig = px.histogram(
        filtered_data, x="SalePrice",
        title="Distribution of Sale Prices",
        nbins=30,
        color_discrete_sequence=["#636EFA"]
    )

    # Simplified Sunburst Chart
    sunburst_fig = px.sunburst(
        filtered_data,
        path=['Neighborhood', 'Sale.Condition'],  # Simplified to two levels
        values='SalePrice',  # Aggregate sale prices for each level
        title="Sales Breakdown by Neighborhood and Condition",
        color='SalePrice',
        color_continuous_scale='Viridis',
        hover_data={'SalePrice': ':,.2f'}  # Format sale price in hover
    )
    sunburst_fig.update_layout(
        margin=dict(t=50, l=0, r=0, b=0),  # Adjust margins
        coloraxis_colorbar=dict(title="Sale Price ($)")  # Add a title to the color bar
    )

    return scatter_fig, bar_fig, time_fig, heatmap_fig, hist_fig, sunburst_fig


# Run the app
if __name__ == '__main__':
    app.run(debug=True)


   Order        PID  MS.SubClass MS.Zoning  Lot.Frontage  Lot.Area Street  \
0      1  526301100           20        RL         141.0     31770   Pave   
1      2  526350040           20        RH          80.0     11622   Pave   
2      3  526351010           20        RL          81.0     14267   Pave   
3      4  526353030           20        RL          93.0     11160   Pave   
4      5  527105010           60        RL          74.0     13830   Pave   

  Alley Lot.Shape Land.Contour  ... Pool.Area Pool.QC  Fence Misc.Feature  \
0   NaN       IR1          Lvl  ...         0     NaN    NaN          NaN   
1   NaN       Reg          Lvl  ...         0     NaN  MnPrv          NaN   
2   NaN       IR1          Lvl  ...         0     NaN    NaN         Gar2   
3   NaN       Reg          Lvl  ...         0     NaN    NaN          NaN   
4   NaN       IR1          Lvl  ...         0     NaN  MnPrv          NaN   

  Misc.Val Mo.Sold Yr.Sold Sale.Type  Sale.Condition  SalePrice  
0       

<IPython.core.display.Javascript object>

# Key Features:

**Neighborhood Filter:**

Select specific neighborhoods for a localized analysis.
Year Filter: Explore housing trends for specific years.
Interactive Visualizations:

**Scatter Plot:** Displays the relationship between living area (Gr.Liv.Area) and sale price, with color representing overall quality (Overall.Qual).

**Bar Chart:** Highlights average sale prices across different neighborhoods.

**Time-Series Plot:** Tracks average sale price trends over the years.

**Correlation Heatmap:** Shows relationships between numerical variables like sale price, living area, and quality.

**Histogram:** Visualizes the distribution of sale prices.

**Sunburst Chart:** Provides a hierarchical breakdown of sales by neighborhood and sale condition.

# Accessibility:

Colorblind-friendly color palettes (e.g., Viridis, Blues).
Fully responsive design using Bootstrap for usability on any device.

# Insights Gained:

**Relationship Between Living Area and Sale Price:**

Larger homes generally have higher sale prices, with quality (Overall.Qual) further influencing price.

**Neighborhood Variations:**

Neighborhoods like StoneBr and NoRidge command higher average sale prices, while neighborhoods like IDOTRR and BrDale are more affordable.

**Temporal Trends:**

Sale prices remained relatively stable across the years but showed minor variations influenced by market conditions.

**Feature Correlations:**

Strong positive correlations between SalePrice and features like Gr.Liv.Area (living area) and Overall.Qual.

# Strengths of the Project:

**Interactivity:** Dynamic filters enable users to focus on specific subsets of the data.

**Comprehensiveness:** A variety of visualizations cover different facets of the dataset, providing a holistic analysis.

**Aesthetic Appeal:** The dark-themed layout with Bootstrap styling ensures a visually pleasing and user-friendly experience.

# Conclusion:

The Ames Housing Dashboard serves as a powerful tool for analyzing residential property sales in Ames, Iowa. It provides valuable insights into the factors influencing housing prices and enables data-driven decision-making for stakeholders such as real estate professionals, homebuyers, and policy analysts. Through its interactive features and clear visualizations, the dashboard effectively bridges the gap between raw data and actionable insights.

# Video link

https://indiana-my.sharepoint.com/:v:/g/personal/shkunch_iu_edu/EUKJ3mtwZKpMgeaPNfiFixkBymJZ1kOl6DG58fO-23vTig?nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJTdHJlYW1XZWJBcHAiLCJyZWZlcnJhbFZpZXciOiJTaGFyZURpYWxvZy1MaW5rIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXcifX0%3D&e=Y7zGVx